In [19]:
import pandas
import openai 
from openai import AsyncOpenAI as OpenAI
from dotenv import load_dotenv, find_dotenv
import os
import sys
import dotenv
import json
import asyncio
import time
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.chains import Chain
from langchain.vectorstores.chroma import Chroma

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.chains import create_qa_with_sources_chain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


In [3]:
# Specify the path to your .env file
env_file_path = '.env'  # Update this with the correct path

# source_path = __file__

# Get the directory of the current file
# source_dir = os.path.dirname(source_path)

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
db = Chroma(
    persist_directory="./chroma",
    embedding_function=OpenAIEmbeddings(model="text-embedding-ada-002"),
)


In [4]:
db

In [6]:
from langchain_core import Document, BaseRetriever
from typing import List

class SimpleRetriever(BaseRetriever):
    docs: List[Document]
    k: int = 5

    def _get_relevant_documents(self, query: str) -> List[Document]:
        """Return the first k documents from the list of documents"""
        return self.docs[:self.k]

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        """(Optional) async native implementation."""
        return self.docs[:self.k]

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x137e72ad0>)

In [7]:
def load_prompt(prompt_file):

        with open(prompt_file, 'r') as file:
                prompt = file.read().replace('\n', '')
                return prompt
    

In [8]:
llm_model = "gpt-3.5-turbo"
user_instruction = "I want a cone"
model = ChatOpenAI(temperature=0.0, model=llm_model)


In [69]:
template_string = load_prompt('./Project/prompts/11.txt')
template_string

'You are an expert rhino3d designer who can work with rhino3d api,the user will give you instructions about an action in rhino and you should give a command of \'RhinoScript\' forrunning in RhinoCommon framework \'RunScript\' method, put the command in a json.Take these steps to make sure you generate the proper script.1. Find the steps needed for the instruction to be executed.2. Use the context to figure out the parameters and steps of the command needed.3. Find the proper measurements based on the users instruction and assign it to the parameters.4. Make sure you are using the correct parameters and correct order of parameters for each command. 5. Genereate the command for each step and fill in the unknown values for parameters with default values.6. structure the commands in a json format.7. If you have any notes or description about the commands, put it in the \'description\' note in the json.The response format should follow the following JSON format. It has to be a json without 

In [70]:
prompt = PromptTemplate(
    input_variables=["context", "input"], template=template_string
)
prompt

PromptTemplate(input_variables=['context', 'input'], template='You are an expert rhino3d designer who can work with rhino3d api,the user will give you instructions about an action in rhino and you should give a command of \'RhinoScript\' forrunning in RhinoCommon framework \'RunScript\' method, put the command in a json.Take these steps to make sure you generate the proper script.1. Find the steps needed for the instruction to be executed.2. Use the context to figure out the parameters and steps of the command needed.3. Find the proper measurements based on the users instruction and assign it to the parameters.4. Make sure you are using the correct parameters and correct order of parameters for each command. 5. Genereate the command for each step and fill in the unknown values for parameters with default values.6. structure the commands in a json format.7. If you have any notes or description about the commands, put it in the \'description\' note in the json.The response format should 

In [75]:
main_chain = LLMChain(
            llm=model, prompt=prompt
        )

In [47]:
doc_prompt = PromptTemplate(
    template="Content: {page_content}\nSource: {source}",
    input_variables=["page_content", "source"],
)

In [49]:
combine_docs_chain = create_stuff_documents_chain(
    model, prompt
)
retrieval_chain = create_retrieval_chain(db.as_retriever(), combine_docs_chain)


In [50]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x137e72ad0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | PromptTemplate(input_variables=['context', 'input'], template='You are an expert rhino3d designer who can work with rhino3d api,the user will give you instructions about an action in rhino and you should give a command of \'RhinoScript\' forrunning in RhinoCommon framework \'RunScript\' method, put the command in a json.Take these steps to make sure you generate the proper script.1. Find the steps needed for the instruction to be executed.2. Use the con

In [51]:
retrieval_chain.invoke({'input': " I want a cone"})

{'input': ' I want a cone',
 'context': [Document(page_content='VolumeMoments \n \n Report the volume moments of inertia of surfaces and polysurfaces. \n W \n \n WalkAbout \n \n Toggle between WalkAbout and normal navigation modes. \n \n WebBrowser \n \n Opens the Web Browser panel. \n \n Weight \n \n Edit the weight of a curve or surface control point. \n \n Weld \n \n Remove creases from a mesh by merging coincident mesh vertices. \n \n WeldEdge \n \n Remove creases from a mesh by merging coincident mesh vertices along selected edges. \n \n WeldVertices \n \n Remove creases from a mesh by merging all selected mesh vertices. \n \n What \n \n Report object properties details. \n \n WireCut \n \n Trim a polysurface with a curve similar to cutting foam with a heated wire. \n \n Worksession \n \n Manage a list of models that can be used as reference geometry. \n X, Y, Z \n \n Zebra \n \n Visually evaluate surface smoothness and continuity using a stripe map. \n \n ZebraOff \n \n Turn off 

In [53]:
from langchain.chains import SequentialChain

In [78]:
command_question_template = """You are an expert rhino3d designer who can work with rhino3d  command line api.
    Based on the user instruction, what is the best "command line" command that can be used to complete this request?
    {input}
"""
command_prompt = PromptTemplate(
template=command_question_template,
input_variables=["input"]
)


chain_one = LLMChain(llm=model, prompt=command_prompt, 
                     output_key="command")

prompt_two = """Write the steps needed for the {command}"""

prompt_two = PromptTemplate(
template=prompt_two,
input_variables=["command"]
)      

chain_two = LLMChain(llm=model, prompt=prompt_two, output_key="steps")


retrieval_chain_2 = create_retrieval_chain(db.as_retriever(), chain_one)

overall_chain = SequentialChain(
    chains=[chain_one, retrieval_chain_2],
    input_variables=["input"],
    # output_variables=["command", "steps", "RhinoScript"],
    verbose=True
)
retrieval_chain_2.invoke({'input': "I want a birthday hat shape"})


{'input': 'I want a birthday hat shape',
 'context': [Document(page_content='H \n \n HandleCurve \n \n Draw chained  Bézier curves  with editing handles. \n \n Hatch \n \n Create a pattern of lines to fill bounding curves. \n \n HatchBase \n \n Set a starting point for existing hatches. \n \n HatchScale \n \n Scale hatch patterns in model and layout space. \n \n HBar \n \n Edit a curve or surface with Bézier curve editing handles. \n \n Heightfield \n \n Create a surface based on gray-scale color values in an image file. \n \n Helix \n \n Draw a helical curve with options for number of turns, pitch, vertical, reverse, and around a curve. \n \n Help \n \n Open the Help topic for the current command. \n \n Hide \n \n Conceal objects from view. \n \n HideInDetail \n \n Conceal objects in a detail view. \n \n HideLayersInDetail \n \n Conceal layers in a detail view. \n \n HidePt \n \n Conceal control and edit points. \n \n HideRenderMesh \n \n Hide the render mesh displayed with the  ShowR